In [1]:
import numpy as np 
import pandas as pd 
import os 

In [2]:
os.listdir()

['106_2020-01-01T0000_2024-05-17T0000.csv',
 '106_2021-01-01T0000_2024-06-04T0000.csv',
 '1_2020-01-01T0000_2024-05-17T0000.csv',
 '22_2021-01-01T0000_2024-06-04T0000.csv',
 '244_2020-01-01T0000_2024-05-17T0000.csv',
 '244_2021-01-01T0000_2024-06-04T0000.csv',
 '251_2021-01-01T0000_2024-12-31T0000.csv',
 '287_2020-01-01T0000_2024-05-17T0000.csv',
 '288_2020-01-01T0000_2024-05-17T0000.csv',
 '319_2021-01-01T0000_2024-06-04T0000.csv',
 '51_2021-01-01T0000_2024-12-31T0000.csv',
 '52_2021-01-01T0000_2024-12-31T0000.csv',
 '93_2021-01-01T0000_2024-06-04T0000.csv',
 'data_2020-01-01T0000_2024-05-17T0000.csv',
 'data_preparing.ipynb',
 'FI',
 'fi_ee.csv',
 'fi_no4.csv',
 'fi_se1.csv',
 'fi_se1_01_08_21.csv',
 'fi_se3.csv',
 'idf',
 'node_modules',
 'Oomi Spot-hintatieto (1_06_2023-30_09_2023).csv',
 'Oomi Spot-hintatieto (1_10_2023-31_10_23).csv',
 'Oomi Spot-hintatieto (1_11_2023-31_12_2023).csv',
 'Oomi Spot-hintatieto (1_1_2023-31_05_2023).csv',
 'Oomi Spot-hintatieto (1_1_2024-31_03_2024)

In [3]:
fi_se1 = pd.concat([pd.read_csv("fi_se1.csv"), pd.read_csv("fi_se1_01_08_21.csv")])
fi_se3 = pd.read_csv("fi_se3.csv")
fi_ee = pd.read_csv("fi_ee.csv")
fi_no4 = pd.read_csv("fi_no4.csv")

# regup = pd.read_csv("244_2020-01-01T0000_2024-05-17T0000.csv")
# regdwn = pd.read_csv("106_2020-01-01T0000_2024-05-17T0000.csv")
# rrs = pd.read_csv("288_2020-01-01T0000_2024-05-17T0000.csv")
# nspin = pd.read_csv("1_2020-01-01T0000_2024-05-17T0000.csv")

regup = pd.read_csv("244_2021-01-01T0000_2024-06-04T0000.csv")
regdwn = pd.read_csv("106_2021-01-01T0000_2024-06-04T0000.csv")
rrs = pd.read_csv("288_2020-01-01T0000_2024-05-17T0000.csv")
nspin = pd.read_csv("52_2021-01-01T0000_2024-12-31T0000.csv")


In [4]:
fi_se1.drop_duplicates(inplace=True)
fi_se3.drop_duplicates(inplace=True)
fi_ee.drop_duplicates(inplace=True)
fi_no4.drop_duplicates(inplace=True)

fi_se1.columns = ["cat", "fi_se1"]
fi_se3.columns = ["cat", "fi_se3"]
fi_ee.columns = ["cat", "fi_ee"]
fi_no4.columns = ["cat", "fi_no4"]

crossborder = fi_se1.merge(fi_se3, on="cat").merge(fi_ee, on="cat").merge(fi_no4, on="cat")

In [5]:
crossborder.columns = ["End time UTC", "SE1", "SE3", "EE", "NO4"]

In [6]:
# replace +00:00 by +
import re
crossborder["End time UTC"] = crossborder["End time UTC"].apply(lambda x: re.sub(r"\+00:00", "+", x))

In [7]:
# format from 01.01.2021+00:0 to 2021-01-01T00:00:00.000Z
crossborder["End time UTC"] = crossborder["End time UTC"].apply(lambda x: re.sub(r"(\d{2})\.(\d{2})\.(\d{4})\+(\d{2}):(\d{2})", r"\3-\2-\1T\4:\5:00.000Z", x))

In [8]:
rrs["End time UTC"]

0        2020-01-01T00:00:00.000Z
1        2020-01-01T01:00:00.000Z
2        2020-01-01T02:00:00.000Z
3        2020-01-01T03:00:00.000Z
4        2020-01-01T04:00:00.000Z
                   ...           
38347    2024-05-16T19:00:00.000Z
38348    2024-05-16T20:00:00.000Z
38349    2024-05-16T21:00:00.000Z
38350    2024-05-16T22:00:00.000Z
38351    2024-05-16T23:00:00.000Z
Name: End time UTC, Length: 38352, dtype: object

In [9]:
crossborder["End time UTC"].apply(lambda x: re.sub("T", ":", re.sub(r":00.000Z", "", x)).strip().replace(".", "-")).iloc[27240]

'13-02-2024 00:00'

In [10]:
pd.to_datetime(crossborder["End time UTC"].apply(lambda x: re.sub("T", " ", re.sub(r":00.000Z", "", x)).strip()).str.replace(".", "-"), format="mixed").dt.date

0        2021-01-01
1        2021-01-01
2        2021-01-01
3        2021-01-01
4        2021-01-01
            ...    
29323    2024-05-17
29324    2024-05-17
29325    2024-05-17
29326    2024-05-17
29327    2024-05-17
Name: End time UTC, Length: 29328, dtype: object

In [11]:
# date format: 01.01.2021+00:00
pd.to_datetime(crossborder["End time UTC"], format="mixed")#.dt.date

0        2021-01-01 00:00:00+00:00
1        2021-01-01 01:00:00+00:00
2        2021-01-01 02:00:00+00:00
3        2021-01-01 03:00:00+00:00
4        2021-01-01 04:00:00+00:00
                   ...            
29323    2024-05-17 19:00:00+00:00
29324    2024-05-17 20:00:00+00:00
29325    2024-05-17 21:00:00+00:00
29326    2024-05-17 22:00:00+00:00
29327    2024-05-17 23:00:00+00:00
Name: End time UTC, Length: 29328, dtype: object

In [12]:
def transform_df(df, name):
    if name != "CROSSBORDER":
        value_col = df.columns[-1]
        df = df.drop("Start time UTC", axis=1)
        df["End time UTC"] = pd.to_datetime(df["End time UTC"])
        # extract Delivery date and hour
        df["Delivery Date"] = df["End time UTC"].dt.date
        df["Hour Ending"] = df["End time UTC"].dt.time.apply(lambda x: x.strftime("%H:%M"))
        df = df.drop("End time UTC", axis=1)
        df[f"{name}"] = df[value_col]
        df = df.drop(value_col, axis=1)
        df["Repeated Hour Flag"] = df.groupby(["Delivery Date", "Hour Ending"])[name].transform("count").apply(lambda x: "Y" if x > 1 else "N")
    else:
        value_cols = df.columns[-4:]
        try:
            df = df.drop("Start time UTC", axis=1)
        except KeyError:
            pass
        df["End time UTC"] = pd.to_datetime(df["End time UTC"].apply(lambda x: re.sub("T", " ", re.sub(r":00.000Z", "", x)).strip()).str.replace(".", "-"), format="mixed")
        # extract Delivery date and hour
        df["Delivery Date"] = df["End time UTC"].dt.date
        df["Hour Ending"] = df["End time UTC"].dt.time.apply(lambda x: x.strftime("%H:%M"))
        df = df.drop("End time UTC", axis=1)
        df[name] = df[value_cols].sum(axis=1)
        df = df.drop(value_cols, axis=1)
        df["Repeated Hour Flag"] = df.groupby(["Delivery Date", "Hour Ending"])[name].transform("count").apply(lambda x: "Y" if x > 1 else "N")    
    return df

In [13]:
nspin = transform_df(nspin, "NSPIN")
regup = transform_df(regup, "REGUP")
regdwn = transform_df(regdwn, "REGDN")
rrs = transform_df(rrs, "RRS")
crossborder = transform_df(crossborder, "CROSSBORDER")

In [14]:
crossborder.drop_duplicates(inplace=True, subset=['Delivery Date', 'Hour Ending'])

In [15]:
crossborder["Repeated Hour Flag"] = "N"

In [16]:
[x.shape for x in [nspin, regup, regdwn, rrs, crossborder]]

[(29833, 4), (29898, 4), (30109, 4), (38352, 4), (29325, 4)]

In [17]:
[(x["Delivery Date"].min(), x["Delivery Date"].max()) for x in [nspin, regup, regdwn, rrs, crossborder]]

[(datetime.date(2020, 12, 31), datetime.date(2024, 6, 4)),
 (datetime.date(2020, 12, 31), datetime.date(2024, 6, 3)),
 (datetime.date(2020, 12, 31), datetime.date(2024, 6, 3)),
 (datetime.date(2020, 1, 1), datetime.date(2024, 5, 16)),
 (datetime.date(2021, 1, 1), datetime.date(2024, 5, 17))]

In [18]:
rrs

,Delivery Date,Hour Ending,RRS,Repeated Hour Flag
0,2020-01-01,00:00,52.2,N
1,2020-01-01,01:00,52.2,N
2,2020-01-01,02:00,52.2,N
3,2020-01-01,03:00,52.2,N
4,2020-01-01,04:00,59.1,N
...,...,...,...,...
38347,2024-05-16,19:00,20.9,N
38348,2024-05-16,20:00,22.1,N
38349,2024-05-16,21:00,22.1,N
38350,2024-05-16,22:00,27.7,N


In [19]:
# concatenate all dataframes on Delivery Date and Hour Ending
df = nspin.merge(regup, on=["Delivery Date", "Hour Ending", "Repeated Hour Flag"])
df = df.merge(regdwn, on=["Delivery Date", "Hour Ending", "Repeated Hour Flag"])
df = df.merge(rrs, on=["Delivery Date", "Hour Ending", "Repeated Hour Flag"])
df = df.merge(crossborder, on=["Delivery Date", "Hour Ending", "Repeated Hour Flag"], how="left").fillna(0)


In [20]:
df.columns

Index(['Delivery Date', 'Hour Ending', 'NSPIN', 'Repeated Hour Flag', 'REGUP',
       'REGDN', 'RRS', 'CROSSBORDER'],
      dtype='object')

In [21]:
df

,Delivery Date,Hour Ending,NSPIN,Repeated Hour Flag,REGUP,REGDN,RRS,CROSSBORDER
0,2020-12-31,22:00,19.20799,N,26.55000,18.50,52.79999,0.00
1,2020-12-31,23:00,15.07600,N,24.07000,18.50,73.29999,0.00
2,2021-01-01,00:00,20.51599,N,24.94999,18.50,82.79999,0.00
3,2021-01-01,01:00,0.00000,N,24.35000,24.35,84.09999,0.00
4,2021-01-01,02:00,0.00000,N,23.98000,23.98,84.09999,0.00
...,...,...,...,...,...,...,...,...
29462,2024-05-16,19:00,25.20000,N,430.00000,91.89,20.90000,236328.84
29463,2024-05-16,20:00,30.60000,N,114.48000,114.48,22.10000,139748.75
29464,2024-05-16,21:00,38.03000,N,84.27000,84.27,22.10000,177703.86
29465,2024-05-16,22:00,38.50000,N,4.30000,4.30,27.70000,140518.84


In [22]:
df = df[['Delivery Date', 'Hour Ending', 'Repeated Hour Flag', 'REGDN','REGUP', 'RRS', 'NSPIN', 'CROSSBORDER']]

In [23]:
df

,Delivery Date,Hour Ending,Repeated Hour Flag,REGDN,REGUP,RRS,NSPIN,CROSSBORDER
0,2020-12-31,22:00,N,18.50,26.55000,52.79999,19.20799,0.00
1,2020-12-31,23:00,N,18.50,24.07000,73.29999,15.07600,0.00
2,2021-01-01,00:00,N,18.50,24.94999,82.79999,20.51599,0.00
3,2021-01-01,01:00,N,24.35,24.35000,84.09999,0.00000,0.00
4,2021-01-01,02:00,N,23.98,23.98000,84.09999,0.00000,0.00
...,...,...,...,...,...,...,...,...
29462,2024-05-16,19:00,N,91.89,430.00000,20.90000,25.20000,236328.84
29463,2024-05-16,20:00,N,114.48,114.48000,22.10000,30.60000,139748.75
29464,2024-05-16,21:00,N,84.27,84.27000,22.10000,38.03000,177703.86
29465,2024-05-16,22:00,N,4.30,4.30000,27.70000,38.50000,140518.84


In [24]:
df.duplicated().sum()

0

In [25]:
df["Repeated Hour Flag"].value_counts()

Repeated Hour Flag
N    29467
Name: count, dtype: int64

In [26]:
def get_data_by_year_and_month(df, year, month_name):
    return df.loc[((pd.to_datetime(df["Delivery Date"]).dt.year==year) & (pd.to_datetime(df["Delivery Date"]).dt.month_name().apply(lambda x: x[:3])==month_name))]

In [27]:
get_data_by_year_and_month(df, 2024, "May").nunique()#.groupby("Delivery Date")["Hour Ending"].count().sort_values()#.shape

Delivery Date          16
Hour Ending            20
Repeated Hour Flag      1
REGDN                 241
REGUP                 263
RRS                    41
NSPIN                 176
CROSSBORDER           279
dtype: int64

In [28]:
get_data_by_year_and_month(df, 2024, "Mar").duplicated().sum()

0

In [29]:
get_data_by_year_and_month(df, 2024, "Mar")["Delivery Date"].unique()

array([datetime.date(2024, 3, 1), datetime.date(2024, 3, 2),
       datetime.date(2024, 3, 3), datetime.date(2024, 3, 4),
       datetime.date(2024, 3, 5), datetime.date(2024, 3, 6),
       datetime.date(2024, 3, 7), datetime.date(2024, 3, 8),
       datetime.date(2024, 3, 9), datetime.date(2024, 3, 10),
       datetime.date(2024, 3, 11), datetime.date(2024, 3, 12),
       datetime.date(2024, 3, 13), datetime.date(2024, 3, 14),
       datetime.date(2024, 3, 15), datetime.date(2024, 3, 16),
       datetime.date(2024, 3, 17), datetime.date(2024, 3, 18),
       datetime.date(2024, 3, 19), datetime.date(2024, 3, 20),
       datetime.date(2024, 3, 21), datetime.date(2024, 3, 22),
       datetime.date(2024, 3, 23), datetime.date(2024, 3, 24),
       datetime.date(2024, 3, 25), datetime.date(2024, 3, 26),
       datetime.date(2024, 3, 27), datetime.date(2024, 3, 28),
       datetime.date(2024, 3, 29), datetime.date(2024, 3, 30),
       datetime.date(2024, 3, 31)], dtype=object)

In [30]:
get_data_by_year_and_month(df, 2024, "Mar").loc[get_data_by_year_and_month(df, 2024, "Mar")["Delivery Date"].astype(str)=="2024-03-31"]

,Delivery Date,Hour Ending,Repeated Hour Flag,REGDN,REGUP,RRS,NSPIN,CROSSBORDER
28442,2024-03-31,00:00,N,42.09,42.09,54.300,10.00,0.00
28443,2024-03-31,01:00,N,42.02,42.02,47.600,0.00,0.00
28444,2024-03-31,02:00,N,42.10,42.10,47.600,0.00,0.00
28445,2024-03-31,03:00,N,43.82,43.82,47.600,0.00,0.00
28446,2024-03-31,04:00,N,46.19,46.19,47.600,11.00,0.00
28447,2024-03-31,05:00,N,44.50,44.57,47.600,10.10,0.00
28448,2024-03-31,06:00,N,35.00,48.92,47.600,10.00,0.00
28449,2024-03-31,07:00,N,35.00,50.04,52.600,10.80,0.00
28450,2024-03-31,08:00,N,43.00,49.31,52.600,13.00,0.00
28451,2024-03-31,09:00,N,42.04,62.00,52.600,12.50,0.00


In [31]:
from datetime import datetime

In [32]:
sub = get_data_by_year_and_month(df, 2023, "Oct")

In [33]:
sub["Delivery Date"].unique()

array([datetime.date(2023, 10, 1), datetime.date(2023, 10, 2),
       datetime.date(2023, 10, 3), datetime.date(2023, 10, 4),
       datetime.date(2023, 10, 5), datetime.date(2023, 10, 6),
       datetime.date(2023, 10, 7), datetime.date(2023, 10, 8),
       datetime.date(2023, 10, 9), datetime.date(2023, 10, 10),
       datetime.date(2023, 10, 11), datetime.date(2023, 10, 12),
       datetime.date(2023, 10, 13), datetime.date(2023, 10, 14),
       datetime.date(2023, 10, 15), datetime.date(2023, 10, 16),
       datetime.date(2023, 10, 17), datetime.date(2023, 10, 18),
       datetime.date(2023, 10, 19), datetime.date(2023, 10, 20),
       datetime.date(2023, 10, 21), datetime.date(2023, 10, 22),
       datetime.date(2023, 10, 23), datetime.date(2023, 10, 24),
       datetime.date(2023, 10, 25), datetime.date(2023, 10, 26),
       datetime.date(2023, 10, 27), datetime.date(2023, 10, 28),
       datetime.date(2023, 10, 29), datetime.date(2023, 10, 30),
       datetime.date(2023, 10, 31)

In [34]:

rrs.loc[rrs["Delivery Date"].astype(str)=="2024-03-09"]

,Delivery Date,Hour Ending,RRS,Repeated Hour Flag
36696,2024-03-09,00:00,51.392,N
36697,2024-03-09,01:00,53.600,N
36698,2024-03-09,02:00,53.600,N
36699,2024-03-09,03:00,53.600,N
36700,2024-03-09,04:00,51.395,N
36701,2024-03-09,05:00,56.100,N
36702,2024-03-09,06:00,56.100,N
36703,2024-03-09,07:00,56.900,N
36704,2024-03-09,08:00,56.900,N
36705,2024-03-09,09:00,53.900,N


In [35]:

crossborder.loc[crossborder["Delivery Date"].astype(str)=="2024-03-09"]

,Delivery Date,Hour Ending,CROSSBORDER,Repeated Hour Flag


In [36]:
# data in sub on 2023-10-29 Delivery Date
sub.loc[sub["Delivery Date"].astype(str)=="2023-10-29"]

,Delivery Date,Hour Ending,Repeated Hour Flag,REGDN,REGUP,RRS,NSPIN,CROSSBORDER
24746,2023-10-29,00:00,N,11.00000,27.19000,62.20000,0.00000,49399.35
24747,2023-10-29,01:00,N,9.00000,25.34000,62.20000,0.00000,43093.05
24748,2023-10-29,02:00,N,5.70000,22.35000,62.20000,0.00000,0.00
24749,2023-10-29,03:00,N,1.50000,21.14999,62.20000,0.00000,0.00
24750,2023-10-29,04:00,N,1.50000,21.94999,62.20000,0.00000,41191.15
24751,2023-10-29,05:00,N,1.22000,22.82999,62.20000,28.00000,42272.23
24752,2023-10-29,06:00,N,1.21000,25.01000,62.20000,34.00000,40050.01
24753,2023-10-29,07:00,N,6.00000,25.53999,62.20000,40.10000,7197.19
24754,2023-10-29,08:00,N,12.00000,24.28000,62.20000,44.50000,0.00
24755,2023-10-29,09:00,N,16.00000,23.93000,62.20000,45.89999,41161.12


In [37]:
get_data_by_year_and_month(df, 2023, "Apr").groupby("Delivery Date")["Hour Ending"].count().unique()

array([24], dtype=int64)

In [38]:
for year in range(2021, 2025, 1):
    sub_df = df[pd.to_datetime(df["Delivery Date"]).dt.year == year]    
    sub_df.to_csv(f"FI/CCP/{year}/rpt.00013091.0000000000000000.20210101080006.DAMASMCPC_{year}.csv", index=False)